In [5]:
import numpy as np

In [6]:
N = 2
dim_x = 2
dim_u = 1

def gen_prediction_matrices(Ad, Bd, N):
    T = np.zeros(((dim_x * (N + 1), dim_x)))
    S = np.zeros(((dim_x * (N + 1), dim_u * N)))
    
    # Condensing
    # Efficient implementation
    power_matricies = []
    power_matricies.append(np.eye(dim_x))
    for k in range(N):
        power_matricies.append(power_matricies[k] @ Ad)
    
    for k in range(N + 1):
        T[k * dim_x:(k + 1) * dim_x, :] = power_matricies[k]
        for j in range(N):
            if k > j:
                S[k * dim_x:(k + 1) * dim_x, j * dim_u:(j + 1) * dim_u] = power_matricies[k - j - 1] @ Bd
    
    return T, S

In [7]:
# Dynamics matrices
Ad = np.array([
    [1., 1.],
    [0., 1.]
])
Bd = np.array([
    [0.],
    [1.]
])
T, S = gen_prediction_matrices(Ad, Bd, N)
print(f"T:\n {T}")
print(f"S:\n {S}")

# T = np.array([
#     [1., 0.],
#     [0., 1.],
#     [1., 1.],
#     [0., 1.],
#     [1., 2.],
#     [0., 1.],
# ])
# S = np.array([
#     [0., 0.],
#     [0., 0.],
#     [0., 0.],
#     [1., 0.],
#     [1., 0.],
#     [1., 1.],
# ])

A = np.array([
    [1., 0.]
])
b = np.array([2.])
C = np.array([[1.], [-1.]])
d = np.ones((2, 1))

A_bar = np.kron(np.eye(N + 1), A)
b_bar = np.kron(np.ones((N + 1, 1)), b)
C_bar = np.kron(np.eye(N), C)
d_bar = np.kron(np.ones((N, 1)), d)

G = A_bar @ T
G = np.vstack([G, np.zeros((C_bar.shape[0], dim_x))])
H = np.vstack((A_bar @ S, C_bar))
psi = -np.vstack((b_bar, d_bar))

print(f"G:\n {G}")
print(f"H:\n {H}")
print(f"psi:\n {psi}")


T:
 [[1. 0.]
 [0. 1.]
 [1. 1.]
 [0. 1.]
 [1. 2.]
 [0. 1.]]
S:
 [[0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 0.]
 [1. 0.]
 [1. 1.]]
G:
 [[1. 0.]
 [1. 1.]
 [1. 2.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
H:
 [[ 0.  0.]
 [ 0.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [-1. -0.]
 [ 0.  1.]
 [-0. -1.]]
psi:
 [[-2.]
 [-2.]
 [-2.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]]


In [8]:
def proj_single_input(G, H, psi):
    # Define the sets by basing on the i-th column of H
    s = psi.shape[0]

    I_0 = []
    I_p = []
    I_m = []

    for i in range(s):
        if H[i] == 0:
            I_0.append(i)
        if H[i] > 0:
            I_p.append(i)
        if H[i] < 0:
            I_m.append(i)

    # Set the row of matrix [P gamma]

    # Define C
    C = np.hstack((G, psi))

    # Define row by row [P gamma]
    aux = []
    for i in I_0:
        aux.append(C[i])

    for i in I_p:
        for j in I_m:
            aux.append(H[i]*C[j] - H[j]*C[i])

    # Return the desired matrix/vector
    aux = np.array(aux)
    P = aux[:,:-1]
    gamma = aux[:,[-1]]

    return P, gamma

def proj_input(G, H, psi, N, dim_u):
    G_i = np.hstack((G, H[:,:-1]))
    H_i = H[:,-1]
    psi_i = psi

    for i in range(N * dim_u, 0, -1):
        P_i, gamma_i = proj_single_input(G_i, H_i, psi_i)

        G_i = P_i[:,:-1]
        H_i = P_i[:,-1]
        psi_i = gamma_i

    return P_i, gamma_i


P, gamma = proj_input(G, H, psi, N, dim_u)

print(f"P:\n {P}")
print(f"gamma:\n {gamma}")


P:
 [[1. 0.]
 [1. 1.]
 [0. 0.]
 [1. 2.]
 [0. 0.]]
gamma:
 [[-2.]
 [-2.]
 [-2.]
 [-3.]
 [-2.]]
